---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
#     for i in range(5):
#         print(nx.average_clustering(P1_Graphs[i]), nx.average_shortest_path_length(P1_Graphs[i]))
    
    graph_types = ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
    return graph_types

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    df = pd.DataFrame(G.nodes(), columns=['nodes'])
    df = df.set_index('nodes')

    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    # Extract node features
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['page_rank'] = pd.Series(nx.pagerank(G, alpha=0.85))
    df['hits_auth'] = pd.Series(nx.hits(G)[1])

    df = (df-df.min())/(df.max()-df.min())

    # Split test and training set
    data = df[~df['ManagementSalary'].isnull()]
    X_test = df[df['ManagementSalary'].isnull()].drop(['ManagementSalary'], axis=1)
    X_train, y_train = data.drop('ManagementSalary', axis=1), data['ManagementSalary'].astype(int)

    clf = RandomForestClassifier(n_estimators=100, max_features=3, max_depth=None)
    clf.fit(X_train, y_train)
    return pd.Series(clf.predict_proba(X_test)[:, 1], index=X_test.index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [17]:
def new_connections_predictions():
    df = future_connections
    def itter_to_series(itterator):
        itter_data = list(itterator)
        data = list(map(lambda x: x[2], itter_data))
        index = list(map(lambda x: (x[0], x[1]), itter_data))
        return pd.Series(data=data, index=index, dtype=float)
    
    df['jaccard_coefficient'] = itter_to_series(nx.jaccard_coefficient(G))
    df['resource_allocation_index'] = itter_to_series(nx.resource_allocation_index(G))
    df['preferential_attachment'] = itter_to_series(nx.preferential_attachment(G))
    
    # Normalize data
    df = (df-df.min())/(df.max()-df.min())

    from sklearn.model_selection import GridSearchCV
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score

    # Split test and training set
    data = df[~df['Future Connection'].isnull()]
    X_test = df[df['Future Connection'].isnull()].drop(['Future Connection'], axis=1)
    X_train, y_train = data.drop('Future Connection', axis=1), data['Future Connection'].astype(int)

    clf = RandomForestClassifier(n_estimators=100)
    param_grid = {
        'max_features': [1,2,3],
        'max_depth': [None,1,2,3,4,5]
    }
    grid = GridSearchCV(clf, param_grid=param_grid)
    clf.fit(X_train, y_train)
    return pd.Series(clf.predict_proba(X_test)[:, 1], index=X_test.index)

In [18]:
# new_connections_predictions()

(107, 348)    0.010000
(542, 751)    0.013486
(20, 426)     0.750000
(50, 989)     0.014662
(942, 986)    0.010877
(324, 857)    0.015456
(13, 710)     0.160000
(19, 271)     0.160000
(319, 878)    0.013238
(659, 707)    0.014140
(49, 843)     0.014470
(208, 893)    0.005920
(377, 469)    0.000000
(405, 999)    0.000000
(129, 740)    0.000000
(292, 618)    0.070000
(239, 689)    0.006051
(359, 373)    0.191696
(53, 523)     0.000000
(276, 984)    0.007716
(202, 997)    0.009551
(604, 619)    0.080000
(270, 911)    0.014375
(261, 481)    0.020000
(200, 450)    0.590000
(213, 634)    0.017184
(644, 735)    0.110000
(346, 553)    0.000000
(521, 738)    0.020359
(422, 953)    0.000000
                ...   
(672, 848)    0.014375
(28, 127)     0.980000
(202, 661)    0.011272
(54, 195)     1.000000
(295, 864)    0.013014
(814, 936)    0.011797
(839, 874)    0.010877
(139, 843)    0.013130
(461, 544)    0.013426
(68, 487)     0.000000
(622, 932)    0.016153
(504, 936)    0.000000
(479, 528) 

In [19]:
# df = future_connections
# df.head()

In [20]:
# def itter_to_series(itterator):
#     itter_data = list(itterator)
#     data = list(map(lambda x: x[2], itter_data))
#     index = list(map(lambda x: (x[0], x[1]), itter_data))
#     return pd.Series(data=data, index=index, dtype=float)

In [21]:
# df['jaccard_coefficient'] = itter_to_series(nx.jaccard_coefficient(G))

In [22]:
# df['resource_allocation_index'] = itter_to_series(nx.resource_allocation_index(G))

In [23]:
# df['preferential_attachment'] = itter_to_series(nx.preferential_attachment(G))

In [24]:
# df.head()

In [25]:
# df = (df-df.min())/(df.max()-df.min())

# # from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score

# # Split test and training set
# data = df[~df['Future Connection'].isnull()]
# X_test = df[df['Future Connection'].isnull()].drop(['Future Connection'], axis=1)
# X_train, y_train = data.drop('Future Connection', axis=1), data['Future Connection'].astype(int)

# clf = RandomForestClassifier(n_estimators=100)
# param_grid = {
#     'max_features': [1,2,3],
#     'max_depth': [None,1,2,3,4,5]
# }
# grid = GridSearchCV(clf, param_grid=param_grid)
# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_train)
# print('roc_auc_score:', roc_auc_score(y_train, y_pred))

# return pd.Series(clf.predict_proba(X_test)[:, 1], index=X_test.index)